# Q1 — Baseline Analysis and Random Forest Regression (Console & Genre → Total Sales)

## Aim
This notebook investigates whether a game’s **console** and **genre** are informative predictors of its **total global sales**.

## Scope (filtered dataset)
To keep the analysis focused and comparable, we restrict the dataset to:
- Consoles: **PS4, XOne, PC**
- Genres: **Action, Sports, Role-Playing, Adventure**

## Approach
1. Load and clean the dataset (remove missing values)
2. Filter to the chosen consoles/genres
3. Compute **baseline mean sales** (simple, interpretable comparison)
4. Train a **Random Forest Regressor** using one-hot encoding
5. Predict which **console–genre combinations** are expected to sell best

In [ ]:
import sys
from pathlib import Path

# Repo root is one level above Q1_folder
ROOT = Path().resolve().parent
sys.path.append(str(ROOT / "py"))

DATA_PATH = ROOT / "data" / "VideoGames_Sales.xlsx"

import seaborn as sns
sns.set(style="whitegrid")

from functions import (
    get_data,
    filter_ps4_xone_pc_and_selected_genres,
    baseline_means,
    encode_console_genre,
    train_and_evaluate_rf,
    top_console_genre_predictions
)

## 1) Load and inspect the data

We load the Excel file, keep only the three columns needed for this coursework question,
and remove rows containing missing values. This ensures the model is trained on complete examples.

In [ ]:
df = get_data(DATA_PATH)

print("Dataset shape (after selecting columns + dropping NaNs):", df.shape)
df.head()

In [ ]:
df.info()

## 2) Filter to the chosen consoles and genres

The coursework question focuses on whether console/genre affect sales.  
To reduce noise and keep comparisons meaningful, we filter to three consoles (PS4, XOne, PC)
and four genres (Action, Sports, Role-Playing, Adventure).

## 3) Baseline: mean sales by category

Before using machine learning, we compute baseline averages:
- Mean sales by **console**
- Mean sales by **genre**
- Mean sales by **console × genre**

This gives a simple reference point to compare with the Random Forest predictions later.

In [ ]:
mean_console, mean_genre, mean_console_genre = baseline_means(df, masks)

print("Mean total sales by console:")
display(mean_console)

print("\nMean total sales by genre:")
display(mean_genre)

print("\nMean total sales by console × genre:")
display(mean_console_genre.head(12))

## 4) Train a Random Forest Regressor

### Why Random Forest?
A Random Forest is a strong baseline model for tabular data because it:
- handles non-linear relationships,
- is robust to noise,
- works well with one-hot encoded categorical features,
- and can outperform simple linear models without heavy tuning.

### Feature representation
Console and genre are categorical, so we use **one-hot encoding** to create numeric input features.

In [ ]:
x_encoded, y = encode_console_genre(F_DataFrame)

model, results = train_and_evaluate_rf(x_encoded, y)
results

### Interpreting the metrics
- **R²** measures how much of the variance in sales is explained by the model (higher is better).
- **RMSE** measures the typical prediction error in the same units as the target variable (lower is better).

Because we only use two features (console + genre), we should not expect perfect performance.
Sales are influenced by many other factors (marketing, franchise popularity, release timing, reviews, etc.).

## 5) Predict the best console–genre combinations

Finally, we create all possible console–genre combinations present in the filtered data,
encode them in the same feature space as training, and predict expected total sales.

In [ ]:
top_combos = top_console_genre_predictions(model, F_DataFrame, x_encoded.columns, top_n=12)
top_combos

In [ ]:
import matplotlib.pyplot as plt

mean_console.plot(kind="bar")
plt.title("Mean Total Sales by Console (Filtered)")
plt.ylabel("total_sales(mil)")
plt.show()

In [ ]:
pivot = mean_console_genre.pivot(index="console", columns="genre", values="total_sales(mil)")
pivot.plot(kind="bar")
plt.title("Mean Total Sales by Console × Genre (Filtered)")
plt.ylabel("total_sales(mil)")
plt.show()

## Conclusion (Q1)

- Baseline means provide an interpretable first look at which consoles and genres tend to sell best on average.
- A Random Forest trained only on **console** and **genre** can capture some predictive structure,
  but performance is limited because many important sales drivers are not included.
- The top predicted console–genre combinations provide an ML-based estimate of what might be expected to sell best,
  which can be compared against the baseline mean rankings.